# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [3]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

/Users/pham/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/pham/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [4]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[int(num_test/2):], newsgroups_test.target[int(num_test/2):]
dev_data, dev_labels = newsgroups_test.data[:int(num_test/2)], newsgroups_test.target[:int(num_test/2)]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print('training label shape:', train_labels.shape)
print('test label shape:', test_labels.shape)
print('dev label shape:', dev_labels.shape)
print('labels names:', newsgroups_train.target_names)

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

[2 pts]

In [5]:
def P1(num_examples=5):

### STUDENT START ###

    for i in range(num_examples):
        print('label: ', i+1, ':', newsgroups_train.target_names[train_labels[i]])
        print('text: ', train_data[i])
        print('-----------------')

### STUDENT END ###
P1()

label:  1 : comp.graphics
text:  Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
-----------------
label:  2 : talk.religion.misc
text:  

Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh have been demonstrating such evi

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

[6 pts]

In [6]:
def P2():
## STUDENT START ###
    cv = CountVectorizer()
    X = cv.fit_transform(train_data)
    
    #Part A
    print('Part A')
    print('Size of Vocabulary: ', X.shape[1])
    print('Average number of non-zero features per example: ', X.nnz/X.shape[0])
    print('Fraction of the entries in the matrix are non-zero: ', X.nnz/(X.shape[0]* X.shape[1]))
    print('----------')
    
    #Part B
    print('Part B')
    print('The 0th feature is: ', cv.get_feature_names()[0] )
    print('The last feature is: ', cv.get_feature_names()[X.shape[1]-1])
    print('---------')
    
    
    #Part C
    print('Part C')
    my_vocab = ["atheism", "graphics", "space", "religion"]
    cv2 = CountVectorizer(vocabulary = my_vocab)
    X2 = cv2.fit_transform(train_data)
    print('Vector Shape: ', X2.shape)
    print('Average number of non-zero features per example:', X2.nnz/X2.shape[0])
    print('---------')
    
       
    #Part D
    print('Part D')
    bigram_cv = CountVectorizer(analyzer = 'char', ngram_range=(2,2))
    bigram_x = bigram_cv.fit_transform(train_data)
    trigram_cv = CountVectorizer(analyzer = 'char', ngram_range=(3,3))
    trigram_x = trigram_cv.fit_transform(train_data)
    print('Bigram size of vocabulary: ', bigram_x.shape[1])
    print('Trigram size of vocabulary: ', trigram_x.shape[1])
    print('-------')
    
    #Part E
    print('Part E')
    prune_cv = CountVectorizer(min_df = 10)
    prune_x = prune_cv.fit_transform(train_data)
    print('Size of Prune Vocab: ', prune_x.shape[1])
    print('-------')


    #Part F
    print('Part F')
    dev_cv = CountVectorizer()
    dev_x = dev_cv.fit_transform(dev_data)
    missing_words = set(dev_cv.get_feature_names()) - set(cv.get_feature_names())
    print('Fraction of words missing: ', len(missing_words)/dev_x.shape[1])

## STUDENT END ###
P2()



# Notes: X gives 2034 rows, and 26879 is the number of rows of words at least 2 letters long



Part A
Size of Vocabulary:  26879
Average number of non-zero features per example:  96.70599803343165
Fraction of the entries in the matrix are non-zero:  0.0035978272269590263
----------
Part B
The 0th feature is:  00
The last feature is:  zyxel
---------
Part C
Vector Shape:  (2034, 4)
Average number of non-zero features per example: 0.26843657817109146
---------
Part D
Bigram size of vocabulary:  3291
Trigram size of vocabulary:  32187
-------
Part E
Size of Prune Vocab:  3064
-------
Part F
Fraction of words missing:  0.24787640034470024


(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

[4 pts]

In [7]:
def P3():
## STUDENT START ###

    # Define vectorizers for train and dev data
    train_cv = CountVectorizer()
    train_X = train_cv.fit_transform(train_data)
    
    dev_cv = CountVectorizer(vocabulary=train_cv.vocabulary_)
    dev_X = dev_cv.fit_transform(dev_data)
    
    # Finding the optimal K for KNN
    print("KNN")
    for k in range(1,10):
        
        knn = KNeighborsClassifier(k)
        knn.fit(train_X, train_labels)
        print("K =", k, "    F1 score =", 
              metrics.f1_score(dev_labels, knn.predict(dev_X), average='micro'))
    print("--------------")
        
    #Find optimal Value for Alpha
    print("Naive Bayes")
    for a in [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]:
        
        MultiNB = MultinomialNB(alpha=a)
        MultiNB.fit(train_X, train_labels)
        print("alpha =", a, "    F1 score =", 
              metrics.f1_score(dev_labels, 
                               MultiNB.predict(dev_X), 
                               average='micro'))
    print("-----------------")
        
        
        
      # Finding the optimal C for logistic regression
    print("Logistic Regression")
    for c in [0.00001, 0.0001, 0.001, 0.01, 0.1,1, 10, 100]:
        
        logReg = LogisticRegression(C=c)
        logReg.fit(train_X, train_labels)
        print("C =", c, "    F1 score =", metrics.f1_score(dev_labels, logReg.predict(dev_X), average='micro'))
        print("Sum of squared weight values:", [sum(logReg.coef_[i,:] ** 2) for i in range(logReg.coef_.shape[0])])
        
    print("----------")    
   
    
## STUDENT END ###
P3()

KNN
K = 1     F1 score = 0.383136094675
K = 2     F1 score = 0.396449704142
K = 3     F1 score = 0.414201183432
K = 4     F1 score = 0.394970414201
K = 5     F1 score = 0.423076923077
K = 6     F1 score = 0.442307692308
K = 7     F1 score = 0.443786982249
K = 8     F1 score = 0.440828402367
K = 9     F1 score = 0.430473372781
--------------
Naive Bayes
alpha = 0.0     F1 score = 0.394970414201
alpha = 0.0001     F1 score = 0.769230769231
alpha = 0.001     F1 score = 0.775147928994
alpha = 0.01     F1 score = 0.779585798817
alpha = 0.1     F1 score = 0.792899408284
alpha = 0.5     F1 score = 0.788461538462
alpha = 1.0     F1 score = 0.781065088757
alpha = 2.0     F1 score = 0.773668639053
alpha = 10.0     F1 score = 0.69674556213
-----------------
Logistic Regression


/Users/pham/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:699: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc) -


C = 1e-05     F1 score = 0.427514792899
Sum of squared weight values: [0.00031350779765462446, 0.00061012653300675864, 0.00038482938888890968, 0.0003414566121614291]
C = 0.0001     F1 score = 0.556213017751
Sum of squared weight values: [0.0077017494467523703, 0.011941200310606562, 0.0094350768656394322, 0.0091028352938318964]
C = 0.001     F1 score = 0.637573964497
Sum of squared weight values: [0.16509345166829453, 0.20095274690581957, 0.18067093754396751, 0.18724278437848996]
C = 0.01     F1 score = 0.67899408284
Sum of squared weight values: [2.5414784670795796, 2.9397093683565245, 2.862468838763121, 2.2500292098346359]
C = 0.1     F1 score = 0.704142011834
Sum of squared weight values: [27.129497327510709, 24.659044263841327, 27.457741536122626, 23.026053110103742]
C = 1     F1 score = 0.701183431953
Sum of squared weight values: [166.98390598771076, 130.90932401664756, 157.94489671189979, 145.73682998189682]
C = 10     F1 score = 0.690828402367
Sum of squared weight values: [586.

ANSWER:
Optimal k for KNN is 7, Optimal alpha for NB is 0.1. Optimal C for logistic regression is 0.1.

(a)KNN doesn't work well here because distance does not explain much about characteristics of the examples. For example, two texts that are unrelated are likely to have close distance as long as they have words in common. 

(b) I am not too sure why, but a hypothesis could be that the training data was not large enough. 

(C) There is a linear relationship. As c increases, so does the sum of squared weights. 


(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

[5 pts]

In [8]:
def P4():
## STUDENT START ###
    
    #function to create 20x4 table based on vectorizer
    def Table(cv):
        
         # Fit data using vectorizer
        train_X = cv.fit_transform(train_data)

        # Fit logistic regression
        logReg = LogisticRegression(C=0.1)
        logReg.fit(train_X, train_labels)

        # Find a list with indexes of 5 features with the largest weights
        # for each label
        indexes = []
        for label in logReg.coef_:
            indexes = np.concatenate((indexes, np.argsort(label)[-5:]))
            
        #remove the decimal
        indexes = [int(indexes[i]) for i in range(len(indexes))]
        
        # Index coefficients using the list created above
        my_table = logReg.coef_[:,indexes]

        # Finding actual words with the largest weights
        words = []
        for i in range(len(indexes)):
            for k, v in cv.vocabulary_.items():
                if v == indexes[i]:
                    words.append(k)

        # Print column names
        print('%20s' % "", end = "")
        for target in newsgroups_train.target_names:
            print('%20s' % target, end = "")
        print("")
        
        # Print row names and table values
        for i in range(len(words)):
            print('%20s' % words[i], end = "")
            for label in range(4):
                print('%20f' % my_table[label,i], end = "")
            print("")
        print("")
        

    # Standard vectorizer
    train_cv = CountVectorizer()
    Table(train_cv)

    # Vectorizer with bigram features
    bigram_cv = CountVectorizer(analyzer='word', ngram_range=(2,2))
    Table(bigram_cv)

## STUDENT END ###
P4()

                             alt.atheism       comp.graphics           sci.space  talk.religion.misc
               islam            0.426283           -0.084818           -0.165049           -0.164898
            atheists            0.461330           -0.079416           -0.158385           -0.295281
               bobby            0.478151           -0.120397           -0.167834           -0.227826
            religion            0.493972           -0.298789           -0.393236            0.003912
             atheism            0.495444           -0.207262           -0.199913           -0.267782
                  3d           -0.181984            0.546999           -0.311629           -0.181435
            computer           -0.039770            0.558989           -0.329038           -0.228708
                file           -0.177353            0.641221           -0.421586           -0.288284
               image           -0.263573            0.642081           -0.367632           

ANSWER: To be honest, I am not sure what I am supposed to be looking for but I guess I see that "cheers kent" is the highest for alt.atheism and talk.religion.misc. This is interesting because comparing Atheist to Religions, you would think something like "jesus christ" would be greater but instead here its "cheers kent"

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

[4 pts]

In [9]:
def empty_preprocessor(s):
    return s

def better_preprocessor(s):
## STUDENT START ###

    # Lowercase everything
    s = s.lower()
    
    # Replace sequences of numbers with *
    s = re.sub(r'[0-9]{2,}', '*', s)

    # Removing non-letter characters
    s = re.sub(r'[^\sa-zA-Z0-9]', '', s)
    
    return s

## STUDENT END ###

def P5():
## STUDENT START ###
    # Fit data using vectorizer with empty_preprocessor
    train_cv = CountVectorizer(preprocessor=empty_preprocessor)
    train_X = train_cv.fit_transform(train_data)
    dev_cv = CountVectorizer(vocabulary=train_cv.vocabulary_)
    dev_X = dev_cv.fit_transform(dev_data)
    
    # Fit logistic regression before preprocessing
    logReg = LogisticRegression(C=0.1)
    logReg.fit(train_X, train_labels)

    # Print F1 score
    print("Before preprocessing, the size of dictionary is", len(train_cv.vocabulary_))
    print("Before preprocessing, F1 score is:", metrics.f1_score(dev_labels, logReg.predict(dev_X), average='micro'))
    
    # Fit data using vectorizer with better_preprocessor
    train_cv = CountVectorizer(preprocessor=better_preprocessor)
    train_X = train_cv.fit_transform(train_data)
    dev_cv = CountVectorizer(vocabulary=train_cv.vocabulary_)
    dev_X = dev_cv.fit_transform(dev_data)
    
    # Fit logistic regression after preprocessing
    logReg = LogisticRegression(C=0.1)
    logReg.fit(train_X, train_labels)

    # Print F1 score
    print("After preprocessing, the size of dictionary is", len(train_cv.vocabulary_))
    print("After preprocessing, F1 score is:", metrics.f1_score(dev_labels, logReg.predict(dev_X), average='micro'))
    
    
## STUDENT END ###
P5()

Before preprocessing, the size of dictionary is 33291
Before preprocessing, F1 score is: 0.665680473373
After preprocessing, the size of dictionary is 27427
After preprocessing, F1 score is: 0.69674556213


(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

[4 pts]

In [31]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###
    
    # Fit data using vectorizer
    train_cv = CountVectorizer()
    train_X = train_cv.fit_transform(train_data)
    dev_cv = CountVectorizer(vocabulary=train_cv.vocabulary_)
    dev_X = dev_cv.fit_transform(dev_data)
    
    # Fit logistic regression with L1 regularization
    logReg = LogisticRegression(penalty='l1')
    logReg.fit(train_X, train_labels)
    coeff = logReg.coef_.flatten()
    print("L1 regularization, number of non-zero weights is:", sum([1 for i in coeff if i > 0]))
  
    # Fit logistic regression with L2 regularization
    logReg = LogisticRegression()
    logReg.fit(train_X, train_labels)
    coeff = logReg.coef_.flatten()
    print("L2 regularization, number of non-zero weights is:", sum([1 for i in coeff if i > 0]))
   
    

    ### STUDENT END ###
P6()

L1 regularization, number of non-zero weights is: 860
L2 regularization, number of non-zero weights is: 44837


(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

[4 pts]

In [32]:
def P7():
    ## STUDENT START ###
    # Fit data using vectorizer
    train_cv = TfidfVectorizer()
    train_X = train_cv.fit_transform(train_data)
    dev_cv = TfidfVectorizer(vocabulary=train_cv.vocabulary_)
    dev_X = dev_cv.fit_transform(dev_data)

    # Fit logistic regression 
    logReg = LogisticRegression(C=100)
    logReg.fit(train_X, train_labels)
    prob = logReg.predict_proba(dev_X)
    
    # Find R values
    r_values = []
    for sample in range(dev_X.shape[0]):
        r = max(prob[sample]) / prob[sample,dev_labels[sample]]
        r_values.append(r)
        
    #print(sorted(range(len(r_values)), key =lambda i: r_values[i]))
    # Find 3 samples with largest R values
    largest_3r = sorted(range(len(r_values)), key=lambda i: r_values[i])[-3:]

    print(largest_3r)
    
    for i in range(len(largest_3r)):
        prob_vec = prob[largest_3r[i]].tolist()
        print("Document", i+1)
        print("Correct Label:", newsgroups_test.target_names[dev_labels[largest_3r[i]]])
        print("Predicted Label:", newsgroups_test.target_names[prob_vec.index(max(prob_vec))])
        print("R Value:", r_values[largest_3r[i]])
        print(dev_data[largest_3r[i]])
        print("-------------------")
    


    ## STUDENT END ###
P7()

[665, 607, 215]
Document 1
Correct Label: talk.religion.misc
Predicted Label: comp.graphics
R Value: 358.759809821
Can anyone provide me a ftp site where I can obtain a online version
of the Book of Mormon. Please email the internet address if possible.
-------------------
Document 2
Correct Label: alt.atheism
Predicted Label: talk.religion.misc
R Value: 386.260491177

The 24 children were, of course, killed by a lone gunman in a second story
window, who fired eight bullets in the space of two seconds...

-------------------
Document 3
Correct Label: talk.religion.misc
Predicted Label: comp.graphics
R Value: 1010.91944794
I am pleased to announce that a *revised version* of _The Easy-to-Read Book
of Mormon_ (former title: _Mormon's Book_) by Lynn Matthews Anderson is now
available through anonymous ftp (see information below). In addition to the
change in title, the revised ETR BOM has been shortened by several pages
(eliminating many extraneous "that's" and "of's"), and many (minor) e

ANSWER:
It looks like the TfidfVectorizer re-weights features by reducing weights of words that are common.

The model looks liek it just looks for keywords without considering the entire passage. This is a problem because the meaning of the passage could be misunderstood when just looking at the keywords. A possible solution could be to look at pairs of words. 

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

- [1 pt] for a reasonable attempt
- [2 pts] for improved performance